# Hands-on!

Nessa prática, sugerimos alguns pequenos exemplos para você implementar sobre o Spark.

## Estimar o Pi

Existe um algoritmo para estimar o Pi com números radômicos. Implemente-o sobre o Spark.

Descrição do algoritmo: http://www.eveandersson.com/pi/monte-carlo-circle

Implementação EM PYTHON (__não sobre o SPARK__): http://www.stealthcopter.com/blog/2009/09/python-calculating-pi-using-random-numbers/

O númer de pontos deve ser 100000 (cem mill) vezes o número mínimo de partições padrão do seu SparkContext (`sc.defaultMinPartitions`). Esses pontos devem ser selecionados aleatóriamente na etapa de map (ver observações).

Observações: use as funções __map__ (para mapear as ocorrêncas em `0` ou `1`, significando `1` quando o ponto aleatório cair dentro do círculo e `0` quando o contrário) e __reduce__ (para sumar as ocorrências).

In [4]:
from math import sqrt
from random import *

In [5]:
def area(x,y):return 1 if sqrt(x*x+y*y)<=1 else  0

def mapf(z):
    x= random()
    y=random()
    return area(x, y)

In [6]:
n = sc.defaultMinPartitions * 100000
data = sc.parallelize(range(1, n))

r = data.map(mapf)

In [7]:
soma = r.reduce(lambda a, b: a+b)
print(soma)
print(r.count())

157336
199999


In [8]:
soma = int(soma)

In [9]:
pi = 4*(soma/(r.count()))
print(pi)

3.1467357336786685


## Filtragem de Primos

Dado uma sequência de números de `1` a `1000000`, filtre somente os primos dessa sequência.

In [10]:
def isPrime(number):
    for i in range(1,number):
        if((i != 1) and (i != number)):
            if ((number % i) == 0):
                return False
    print(number)
    return True

In [11]:
data = sc.parallelize(range(1, 1000000))

r = data.filter(isPrime)

print(r.take(100))

[1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523]


## Municípios do Brasil

Dado o dataset `mucipios_do_Brasil.csv`, faça duas operações com ele:

1. Monte uma lista dos municípios por estado.
2. Conte quantos municípios há em cada estado.

Dicas: use as operações groupByKey e reduceByKey, não faça um count na lista da operação 1. 

In [12]:
file = sc.textFile('municipios_do_Brasil.csv')

In [13]:
file_mapping = file.map(lambda line: line.split(',')).map(lambda word: (word[0], word[1]))

In [14]:
#Contando o numero de municipios por Estado
count_cities = file_mapping.map(lambda word: (word[0], 1)).reduceByKey(lambda x, y: x + y)

print(count_cities.collect())

[('AC', 22), ('AP', 16), ('BA', 417), ('CE', 184), ('DF', 1), ('ES', 78), ('MG', 853), ('PE', 185), ('PR', 399), ('RJ', 92), ('RN', 167), ('SC', 293), ('SP', 645), ('TO', 137), ('uf', 1), ('AL', 102), ('AM', 62), ('GO', 246), ('MA', 217), ('MS', 78), ('MT', 141), ('PA', 143), ('PB', 223), ('PI', 224), ('RO', 67), ('RS', 496), ('SE', 75)]


In [15]:
#Listando os municipios por Estado
list_cities = file_mapping.groupByKey().map(lambda x : (x[0], list(x[1])))

print(list_cities.take(1))

[('AC', ['Acrelândia', 'Assis Brasil', 'Brasiléia', 'Bujari', 'Capixaba', 'Cruzeiro do Sul', 'Epitaciolândia', 'Feijó', 'Jordão', 'Mâncio Lima', 'Manoel Urbano', 'Marechal Thaumaturgo', 'Plácido de Castro', 'Porto Acre', 'Porto Walter', 'Rio Branco', 'Rodrigues Alves', 'Santa Rosa do Purus', 'Sena Madureira', 'Senador Guiomard', 'Tarauacá', 'Xapuri'])]


## Word Count Memória Postumas de Brás Cubas

Memórias Póstumas de Brás Cubas é um romance escrito por Machado de Assis, desenvolvido em princípio como folhetim, de março a dezembro de 1880, na Revista Brasileira, para, no ano seguinte, ser publicado como livro, pela então Tipografia Nacional.

A obra retrata a escravidão, as classes sociais, o cientificismo e o positivismo da época. Dada essas informações, será que conseguimos idenficar essas características pelas palavras mais utilizadas em sua obra?

Utilizando o dataset `Machado-de-Assis-Memorias-Postumas.txt`, faça um word count e encontre as palavras mais utilizadas por Machado de Assis em sua obra. Não esqueça de utilizar `stopwords.pt` para remover as `stop words`!

In [16]:
file2 = sc.textFile('Machado-de-Assis-Memorias-Postumas.txt')

In [41]:
stopwords = sc.textFile('stopwords.pt')
stopwords = stopwords.flatMap(lambda word: word.split())
stopwords = stopwords.collect()

In [42]:
stopwords

['(', ')', '?', '!', '"', ';', '|', '[', ']', '.', ',', "'", '-', '+', '-', '*', '{', '}', '&', '%', '$', '#', '@', ':', '/', 'a', 'à', 'acordo', 'afirma', 'afirmou', 'agora', 'ainda', 'além', 'alguns', 'ano', 'anos', 'antes', 'ao', 'aos', 'apenas', 'após', 'aqui', 'área', 'as', 'às', 'assim', 'até', 'aumento', 'banco', 'bem', 'bilhões', 'bom', 'brasil', 'brasileira', 'brasileiro', 'brasília', 'cada', 'câmara', 'campanha', 'candidato', 'carlos', 'casa', 'caso', 'central', 'centro', 'cerca', 'cidade', 'cinco', 'cinema', 'coisa', 'com', 'como', 'congresso', 'conta', 'contra', 'da', 'dar', 'das', 'de', 'depois', 'deputado', 'desde', 'deve', 'dia', 'dias', 'dinheiro', 'direito', 'diretor', 'disse', 'diz', 'do', 'dois', 'dos', 'duas', 'durante', 'e', 'é', 'economia', 'econômica', 'ela', 'ele', 'eles', 'em', 'empresa', 'empresas', 'enquanto', 'então', 'entre', 'equipe', 'era', 'especial', 'essa', 'esse', 'esta', 'está', 'estado', 'estados', 'estão', 'estava', 'este', 'eu', 'eua', 'exemplo', 

In [54]:
def filter_words(word):
    print(word[0])
    if word[0] in stopwords:
        return False
    return True

In [59]:
counts = file2.flatMap(lambda line: line.split(" ")).map(lambda word: word.lower()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
counts = counts.filter(filter_words)
counts = counts.takeOrdered(100, key = lambda x: -x[1])

In [60]:
print(counts)

[('—', 723), ('', 303), ('capítulo', 174), ('lhe', 159), ('virgília', 111), ('olhos', 102), ('d.', 85), ('alguma', 81), ('talvez', 74), ('que,', 68), ('ia', 68), ('homem', 65), ('quincas', 65), ('podia', 60), ('fui', 58), ('dizer', 57), ('e,', 55), ('sei', 54), ('lá', 54), ('eram', 54), ('lobo', 52), ('logo', 50), ('algum', 49), ('isto', 49), ('meus', 48), ('dizia', 48), ('nossa', 46), ('idéia', 46), ('olhar', 45), ('mim', 45), ('eu,', 44), ('tal', 41), ('coisas', 41), ('fosse', 41), ('virgília,', 41), ('certo', 40), ('mão', 40), ('depois,', 38), ('modo', 38), ('nada.', 37), ('te', 37), ('simples', 37), ('tempo,', 36), ('uns', 35), ('ver', 35), ('pai', 35), ('creio', 35), ('aí', 34), ('não,', 33), ('ir', 33), ('sim,', 33), ('brás', 33), ('algumas', 33), ('nunca', 33), ('si', 32), ('marcela', 32), ('mim,', 31), ('digo', 30), ('ele,', 30), ('certa', 30), ('morte', 29), ('nenhum', 29), ('pé', 29), ('mas,', 29), ('tu', 29), ('anos,', 28), ('nosso', 28), ('muitas', 28), ('borba', 28), ('dis